In [1]:
import warnings
import pandas as pd
import geopandas as gpd
import glob
import numpy as np
# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.5f}'.format

# 데이터 불러오기

In [1]:
import pandas as pd

df = pd.read_csv('./final/pre_melt.csv')

# 면적 구하기

In [3]:
from shapely.wkt import loads

# Assuming df['geometry'] contains a WKT-formatted POLYGON string
df['area'] = df['geometry'].apply(lambda geom_str: loads(geom_str).area if geom_str is not None else None)

In [4]:
from shapely.wkt import loads

# Assuming df['geometry'] contains a WKT-formatted POLYGON string
df['area_m2'] = df['geometry'].apply(lambda geom_str: loads(geom_str).area * (111.32**2) * (10**6) if geom_str is not None else None)

In [5]:
from shapely.wkt import loads

# Assuming df['geometry'] contains a WKT-formatted POLYGON string
df['area_km2'] = df['geometry'].apply(lambda geom_str: loads(geom_str).area * (111.32**2) if geom_str is not None else None)

# 필요 컬럼 추출

In [6]:
df_l = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', 'cluster']]
df_r = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_상주인구_수', '연령대_40_상주인구_수', '연령대_50_상주인구_수', '연령대_60_이상_상주인구_수', 'cluster']]
df_w = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '연령대_10_직장_인구_수', '연령대_20_직장_인구_수', '연령대_30_직장_인구_수', '연령대_40_직장_인구_수', '연령대_50_직장_인구_수', '연령대_60_이상_직장_인구_수', 'cluster']]
df_f = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '집객시설수', '교통시설수', 'cluster']]
df_s = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '유사_업종_점포_수', 'cluster']]
df_e = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', 'cluster']]

# 컬럼명 변경 & melt

In [7]:
df_l = df_l.rename(columns = {'연령대_10_유동인구_수' : '10대',
                              '연령대_20_유동인구_수' : '20대',
                              '연령대_30_유동인구_수' : '30대',
                              '연령대_40_유동인구_수' : '40대',
                              '연령대_50_유동인구_수' : '50대',
                              '연령대_60_이상_유동인구_수' : '60대 이상'})

df_l = pd.melt(df_l, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster', 'area_m2'],
                var_name = '연령', value_name = '유동인구수')

In [9]:
df_r = df_r.rename(columns = {'연령대_10_상주인구_수' : '10대',
                              '연령대_20_상주인구_수' : '20대',
                              '연령대_30_상주인구_수' : '30대',
                              '연령대_40_상주인구_수' : '40대',
                              '연령대_50_상주인구_수' : '50대',
                              '연령대_60_이상_상주인구_수' : '60대 이상'})
df_r = pd.melt(df_r, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster', 'area_m2'],
                var_name = '연령', value_name = '상주인구수')

In [11]:
df_w = df_w.rename(columns = {'연령대_10_직장_인구_수' : '10대',
                              '연령대_20_직장_인구_수' : '20대',
                              '연령대_30_직장_인구_수' : '30대',
                              '연령대_40_직장_인구_수' : '40대',
                              '연령대_50_직장_인구_수' : '50대',
                              '연령대_60_이상_직장_인구_수' : '60대 이상'})

df_w = pd.melt(df_w, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster', 'area_m2'],
                var_name = '연령', value_name = '직장인구수')

In [13]:
df_e = df_e.rename(columns = {'연령대_10_매출_금액' : '10대',
                              '연령대_20_매출_금액' : '20대',
                              '연령대_30_매출_금액' : '30대',
                              '연령대_40_매출_금액' : '40대',
                              '연령대_50_매출_금액' : '50대',
                              '연령대_60_이상_매출_금액' : '60대 이상'})

df_e = pd.melt(df_e, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster', 'area_m2'],
                var_name = '연령', value_name = '매출금액')

# 데이터 병합

In [15]:
melt_df = pd.merge(df_l, df_r)
melt_df = pd.merge(melt_df, df_w)
melt_df = pd.merge(melt_df, df_f)
melt_df = pd.merge(melt_df, df_s)
melt_df = pd.merge(melt_df, df_e)

# 단위 면적 구하기

In [17]:
original_col = ['유동인구수', '상주인구수', '직장인구수', '집객시설수', '교통시설수', '유사_업종_점포_수', '매출금액']
area_col = ['단위_유동인구수', '단위_상주인구수', '단위_직장인구수', '단위_집객시설수', '단위_교통시설수', '단위_유사_업종_점포_수', '단위_매출금액']

for i, j in zip(original_col, area_col) :
    for k in range(len(melt_df)):
        if melt_df.loc[k,i] == 0:
            melt_df.loc[k,j] = melt_df.loc[k,i]
        else :
            melt_df.loc[k,j] = melt_df.loc[k,i] / melt_df.loc[k,'area_m2']

# 표준화 시킨 면적 구하기

In [19]:
from sklearn.preprocessing import MinMaxScaler

trans_col = ['정규_유동인구수', '정규_상주인구수', '정규_직장인구수', '정규_집객시설수', '정규_교통시설수', '정규_유사_업종_점포_수', '정규_매출금액']

scaler = MinMaxScaler()
melt_df[trans_col] = scaler.fit_transform(melt_df[area_col])

In [21]:
melt_df.to_csv("./final/melt_df_1.csv", index = False)